In [ ]:
import multiprocessing
from collections import Counter
import time
import re
import chardet
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
import matplotlib.pyplot as plt

# Загружаем стоп-слова один раз (за пределами функций)
try:
    nltk.data.find('corpora/stopwords')
    stop_words = set(stopwords.words('russian'))
except LookupError:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('russian'))


# Функция для чтения файла
def read_file(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read()
    encoding = chardet.detect(raw_data)['encoding']
    with open(file_path, 'r', encoding=encoding or 'utf-8', errors='ignore') as f:
        return f.read()


# Функция для подсчета слов (с фильтрацией)
def word_count(text, stop_words):
    words = re.findall(r'\b\w+\b', text.lower())
    filtered_words = [word for word in words if word not in stop_words]
    return Counter(filtered_words)


# Функция для подсчета фраз
def phrase_count(text, n, stop_words):
    words = re.findall(r'\b\w+\b', text.lower())
    filtered_words = [word for word in words if word not in stop_words]
    phrases = ngrams(filtered_words, n)
    return Counter(phrases)


# Функция-обработчик для подсчета слов/фраз в процессе
def process_text(args):
    part, stop_words, count_type, n = args
    if count_type == "word":
        return word_count(part, stop_words)
    elif count_type == "phrase":
        return phrase_count(part, n, stop_words)


# Основная функция для многопоточной обработки
def parallel_count(file_path, num_processes, count_type="word", n=2):
    text = read_file(file_path)
    text_length = len(text)
    chunk_size = text_length // num_processes
    chunks = [text[i:i + chunk_size] for i in range(0, text_length, chunk_size)]

    args_list = [(chunk, stop_words, count_type, n) for chunk in chunks]

    with multiprocessing.Pool(processes=num_processes) as pool:
        results = pool.map(process_text, args_list)

    total_count = Counter()
    for count in results:
        total_count.update(count)
    return total_count


# Функция для последовательной обработки
def sequential_count(file_path, count_type="word", n=2):
    text = read_file(file_path)
    if count_type == "word":
        return word_count(text, stop_words)
    elif count_type == "phrase":
         return phrase_count(text, n, stop_words)

def visualize_results(results, title, top_n=10):
    if not results:
        print("No results to visualize.")
        return

    most_common = results.most_common(top_n)
    if isinstance(most_common[0][0], tuple): # для фраз
         labels = [' '.join(phrase) for phrase, _ in most_common]
    else: # для слов
        labels = [word for word, _ in most_common]
    values = [count for _, count in most_common]

    plt.figure(figsize=(10,6))
    plt.bar(labels, values)
    plt.xlabel("Words/Phrases")
    plt.ylabel("Frequency")
    plt.title(title)
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    FILE_PATH = "109472392.txt"  # Замените путь к вашему файлу
    NUM_PROCESSES = 4
    
    # Последовательный подсчет слов
    start_time = time.time()
    seq_result_words = sequential_count(FILE_PATH, count_type="word")
    seq_time_words = time.time() - start_time
    print(f"Последовательный подсчет слов занял {seq_time_words:.2f} секунд.")

    # Параллельный подсчет слов
    start_time = time.time()
    parallel_result_words = parallel_count(FILE_PATH, NUM_PROCESSES, count_type="word")
    parallel_time_words = time.time() - start_time
    print(f"Параллельный подсчет слов занял {parallel_time_words:.2f} секунд.")

    print(f"Ускорение (слова): {seq_time_words / parallel_time_words:.2f}x")

    # Топ 10 самых частых слов
    print("\nТоп 10 самых частых слов:")
    for word, count in parallel_result_words.most_common(10):
        print(f"{word}: {count}")

    # Визуализируем результаты подсчета слов
    visualize_results(parallel_result_words, "Топ 10 самых частых слов")
    # Последовательный подсчет фраз (биграмм)
    start_time = time.time()
    seq_result_phrases = sequential_count(FILE_PATH, count_type="phrase", n=2)
    seq_time_phrases = time.time() - start_time
    print(f"\nПоследовательный подсчет фраз (биграмм) занял {seq_time_phrases:.2f} секунд.")

     # Параллельный подсчет фраз (биграмм)
    start_time = time.time()
    parallel_result_phrases = parallel_count(FILE_PATH, NUM_PROCESSES, count_type="phrase", n=2)
    parallel_time_phrases = time.time() - start_time
    print(f"Параллельный подсчет фраз (биграмм) занял {parallel_time_phrases:.2f} секунд.")

    print(f"Ускорение (фразы): {seq_time_phrases / parallel_time_phrases:.2f}x")

    # Топ 10 самых частых фраз (биграмм)
    print("\nТоп 10 самых частых фраз (биграмм):")
    for phrase, count in parallel_result_phrases.most_common(10):
        print(f"{' '.join(phrase)}: {count}")

    # Визуализируем результаты подсчета фраз
    visualize_results(parallel_result_phrases, "Топ 10 самых частых фраз (биграмм)")